In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
"""
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women) / len(women)

print( "% of women who survived: ", rate_women)
"""

'\nwomen = train_data.loc[train_data.Sex == \'female\']["Survived"]\nrate_women = sum(women) / len(women)\n\nprint( "% of women who survived: ", rate_women)\n'

In [5]:
"""
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)
"""

'\nmen = train_data.loc[train_data.Sex == \'male\']["Survived"]\nrate_men = sum(men)/len(men)\n\nprint("% of men who survived:", rate_men)\n'

In [6]:
# ver 1: from sklearn.ensemble import RandomForestClassifier
# ver 2: from sklearn.linear_model import LogisticRegression
# ver 3: from sklearn.linear_model import SGDClassifier
# ver 4: from sklearn.neighbors import KNeighborsClassifier
# ver 5: from sklearn.svm import SVC
# ver 6: from sklearn.tree import DecisionTreeClassifier

"""
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = DecisionTreeClassifier(random_state=0)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
"""

'\ny = train_data["Survived"]\n\nfeatures = ["Pclass", "Sex", "SibSp", "Parch"]\nX = pd.get_dummies(train_data[features])\nX_test = pd.get_dummies(test_data[features])\n\nmodel = DecisionTreeClassifier(random_state=0)\nmodel.fit(X, y)\npredictions = model.predict(X_test)\n\noutput = pd.DataFrame({\'PassengerId\': test_data.PassengerId, \'Survived\': predictions})\noutput.to_csv(\'submission.csv\', index=False)\nprint("Your submission was successfully saved!")\n'

In [7]:
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [8]:
train_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
#inplace=True는 원본 데이터를 수정하겠다는것을 의미

In [9]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked'],
      dtype='object')

In [10]:
sex = pd.get_dummies(train_data['Sex'], drop_first=True)
embark = pd.get_dummies(train_data['Embarked'], drop_first=True)
# drop_first=True 는 첫번째 옵션을 drop함을 뜻함
# 예로 Sex column에서 female이 1이라면, 0인 row는 자동으로 male을 의미
train_data = pd.concat([train_data, sex, embark], axis=1)
train_data.drop(['Sex', 'Embarked'], axis=1, inplace=True)

In [11]:
sex = pd.get_dummies(test_data['Sex'], drop_first=True)
embark = pd.get_dummies(test_data['Embarked'], drop_first=True)
test_data = pd.concat([test_data, sex, embark], axis=1)
test_data.drop(['Sex', 'Embarked'], axis=1, inplace=True)

In [12]:
train_data.fillna(train_data.mean(), inplace=True)
test_data.fillna(test_data.mean(), inplace=True)

In [13]:
y_train = train_data['Survived'].values
sub_PassengerId = test_data['PassengerId'].values

Scaler1 = StandardScaler()
Scaler2 = StandardScaler()

train_columns = train_data.columns
test_columns = test_data.columns

train_data = pd.DataFrame(Scaler1.fit_transform(train_data))
test_data = pd.DataFrame(Scaler2.fit_transform(test_data))

train_data.head()

,0,1,2,3,4,5,6,7,8,9
0,-1.730108,-0.789272,0.827377,-0.592481,0.432793,-0.473674,-0.502445,0.737695,-0.307562,0.619306
1,-1.726220,1.266990,-1.566107,0.638789,0.432793,-0.473674,0.786845,-1.355574,-0.307562,-1.614710
2,-1.722332,1.266990,0.827377,-0.284663,-0.474545,-0.473674,-0.488854,-1.355574,-0.307562,0.619306
3,-1.718444,1.266990,-1.566107,0.407926,0.432793,-0.473674,0.420730,-1.355574,-0.307562,0.619306
4,-1.714556,-0.789272,0.827377,0.407926,-0.474545,-0.473674,-0.486337,0.737695,-0.307562,0.619306


In [14]:
train_data.columns = train_columns
test_data.columns = test_columns

X_train = train_data.iloc[:,2:].values

In [15]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        hidden_1 = 256
        hidden_2 = 128
        hidden_3 = 64
        hidden_4 = 16

        self.fc1 = nn.Linear(8, hidden_1)
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(hidden_2, hidden_3)
        self.dropout3 = nn.Dropout(0.2)
        
        self.fc4 = nn.Linear(hidden_3, hidden_4)
        self.dropout4 = nn.Dropout(0.2)
        
        self.fc5 = nn.Linear(hidden_4, 2)

    def forward(self, x):
        x = x.view(-1, 8)

        x = F.leaky_relu(self.fc1(x))
        x = self.dropout1(x)
        
        x = F.leaky_relu(self.fc2(x))
        x = self.dropout2(x)
        
        x = F.leaky_relu(self.fc3(x))
        x = self.dropout3(x)
        
        x = F.leaky_relu(self.fc4(x))
        x = self.dropout4(x)
        
        x = torch.tanh(self.fc5(x))
        return x

# The model kept overfitting, so I used leaky-ReLU and dropout,
# but the probability is still low.
# It seems like it's just a matter of luck that the accuracy goes up as I reduce the amount of training.
    
# 모델 초기화
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=8, out_features=256, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (dropout3): Dropout(p=0.2, inplace=False)
  (fc4): Linear(in_features=64, out_features=16, bias=True)
  (dropout4): Dropout(p=0.2, inplace=False)
  (fc5): Linear(in_features=16, out_features=2, bias=True)
)


In [16]:
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)

dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [18]:
# I recommend to train less than 50 epochs
# because, if you train more than 100, it always overfits

nb_epochs = 50
train_loss_min = np.Inf
for epoch in range(nb_epochs):
    num_right = 0
    train_loss = 0
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        
        prediction = model(x_train)
        loss = criterion(prediction, y_train.long())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        labels = torch.argmax(prediction, dim=1)
        num_right += torch.sum(labels == y_train)
        train_loss += loss.item() * len(x_train)
        
    train_loss = train_loss / len(X_train)
    if (epoch + 1) % 20 == 0:
        print('Epoch {}/{}, Prediction : {}/{}, Cost : {}'.format(epoch+1, nb_epochs, num_right, len(X_train), train_loss))
    if train_loss <= train_loss_min:
        print('=*=*=*= Loss decreased ({:6f} ===> {:6f}). Saving the model! =*=*=*='.format(train_loss_min, train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss
        
print('Training Ended!')

# If anything, the smaller number of epochs increases the accuracy a bit... wth

=*=*=*= Loss decreased (   inf ===> 0.720177). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.720177 ===> 0.712331). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.712331 ===> 0.702678). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.702678 ===> 0.690869). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.690869 ===> 0.676016). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.676016 ===> 0.656823). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.656823 ===> 0.633988). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.633988 ===> 0.606326). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.606326 ===> 0.587327). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.587327 ===> 0.555628). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.555628 ===> 0.538092). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.538092 ===> 0.525376). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.525376 ===> 0.505300). Saving the model! =*=*=*=
=*=*=*= Loss decreased (0.5

In [19]:
X_test = torch.FloatTensor(test_data.iloc[:,1:].values)
with torch.no_grad():
    result = model(X_test)
labels = torch.argmax(result, dim=1)
survived = labels.numpy()

In [20]:
submission = pd.DataFrame({'PassengerId': sub_PassengerId, 'Survived': survived})
submission.to_csv('submission.csv', index=False)